# 1. Environment Setup

In [1]:
import math
import numpy as np
import pandas as pd

# 2. Data Acquisition

### 2.1 Variables Definition

In [2]:
alloy = ['all_1','all_2','all_3','all_4','all_5']
supplier = ['sup_1','sup_2','sup_3','sup_4','sup_5']
product = ['prod_1','prod_2','prod_3']
month = ['Init', 'Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']
I = len(alloy)
J = len(supplier)
Z = len(product)
M = len(month)
q = 0.05
mu = 1

### 2.2 Data Filepath

In [3]:
file_loc = 'Data Templates_v4.xlsx'

### 2.3 Contract Upper Threshold

In [4]:
contract_upper_threshold = pd.read_excel(file_loc,sheet_name='Contract Upper Threshold',header=2, na_values=['NA'], usecols="B:F")
contract_upper_threshold.index=alloy

In [5]:
h = contract_upper_threshold
contract_upper_threshold

,Supp A,Supp B,Supp C,Supp D,Supp E
all_1,1500,1000,1500,1000,1200
all_2,10000,2500,7500,7500,5000
all_3,10000,3000,6000,4500,1800
all_4,450,250,450,300,100
all_5,500,250,450,300,200


### 2.4 Contract Lower Threshold

In [6]:
contract_lower_threshold = pd.read_excel(file_loc,sheet_name='Contract Lower Threshold',header=2, na_values=['NA'], usecols="B:F")
contract_lower_threshold.index=alloy

In [7]:
l = contract_lower_threshold
contract_lower_threshold

,Supp A,Supp B,Supp C,Supp D,Supp E
all_1,300,400,450,550,400
all_2,5000,800,1800,1000,420
all_3,1500,1200,2500,2000,800
all_4,42,100,85,100,75
all_5,84,100,100,120,50


### 2.5 Contract Penalties

In [8]:
contract_penalties = pd.read_excel(file_loc,sheet_name='Contract Penalties',header=2, na_values=['NA'], usecols="B:F")
contract_penalties.index=['Penalty Fee %']

In [9]:
p = contract_penalties
contract_penalties

,Supp A,Supp B,Supp C,Supp D,Supp E
Penalty Fee %,0.8,0.6,0.75,0.7,0.5


### 2.6 Annual Contract Limit

In [10]:
annual_contracted_limit= pd.read_excel(file_loc,sheet_name='Annual Contracted Limit',header=2, na_values=['NA'], usecols="B:F")
annual_contracted_limit = annual_contracted_limit.iloc[:5,:]
annual_contracted_limit.index=alloy

In [11]:
k = annual_contracted_limit
annual_contracted_limit

,Supp A,Supp B,Supp C,Supp D,Supp E
all_1,9616.2,6956.5,7871.0,7867.8,11807.5
all_2,67200.0,33600.0,22400.0,44800.0,56000.0
all_3,17600.0,44000.0,35200.0,35200.0,44000.0
all_4,1560.0,1872.0,1768.0,3120.0,2080.0
all_5,1356.0,3616.0,2938.0,1130.0,2260.0


### 2.7 Unit Cost

In [12]:
unit_cost = pd.read_excel(file_loc,sheet_name='Unit Cost',header=2, na_values=['NA'], usecols="B:F")
unit_cost = unit_cost.iloc[:5,:]
unit_cost.index=alloy

In [13]:
c = unit_cost
unit_cost

,Supp A,Supp B,Supp C,Supp D,Supp E
all_1,275.000000,274.166667,274.583333,274.416667,273.750000
all_2,11.250000,10.666667,11.083333,10.833333,10.416667
all_3,2.333333,1.833333,2.166667,2.000000,1.666667
all_4,858.333333,855.000000,857.083333,855.833333,853.333333
all_5,210.833333,204.166667,208.333333,205.833333,202.500000


### 2.8 Product Revenue

In [14]:
prod_rev = pd.read_excel(file_loc,sheet_name='Prod Rev',header=2, na_values=['NA'], usecols="B:F")
prod_rev.index=['Revenue ($)']

D:\anaconda\lib\site-packages\pandas\io\excel\_base.py:1272: FutureWarning: Defining usecols with out of bounds indices is deprecated and will raise a ParserError in a future version.
  return self._reader.parse(


In [15]:
r = prod_rev
prod_rev

,Prod 1,Prod 2,Prod 3
Revenue ($),20000,22000,25000


### 2.9 Product Recipe

In [16]:
prod_recipe= pd.read_excel(file_loc,sheet_name='Prod Recipe',header=2, na_values=['NA'], usecols="B:D")
prod_recipe = prod_recipe.iloc[:5,:]
prod_recipe.index=alloy

In [17]:
u = prod_recipe
prod_recipe

,Prod 1,Prod 2,Prod 3
all_1,26.226,49.670455,0.000000
all_2,112.000,152.727273,81.454545
all_3,88.000,40.000000,256.000000
all_4,4.160,11.818182,7.563636
all_5,0.000,15.409091,12.327273


### 2.10 Predicted Demand

In [18]:
predicted_demand= pd.read_excel(file_loc,sheet_name='Predicted Demand',header=2, na_values=['NA'], usecols="B:D")
predicted_demand.index=month

In [19]:
d = predicted_demand
predicted_demand

,Prod 1,Prod 2,Prod 3
Init,0.0,0.0,0.0
Jan,48.4,11.0,13.2
Feb,44.0,13.2,16.5
Mar,33.0,15.4,16.5
Apr,35.2,11.0,16.5
May,39.6,16.5,22.0
Jun,44.0,27.5,27.5
Jul,39.6,24.2,35.2
Aug,44.0,22.0,30.8
Sep,35.2,19.8,24.2


# 3. Baseline Model

In [20]:
alloy_demand = predicted_demand @ prod_recipe.T
alloy_demand

,all_1,all_2,all_3,all_4,all_5
Init,0.0000,0.0,0.0,0.000,0.00
Jan,1815.7134,8176.0,8078.4,431.184,332.22
Feb,1809.5940,8288.0,8624.0,463.840,406.80
Mar,1630.3830,7392.0,7744.0,444.080,440.70
Apr,1469.5302,6966.4,7761.6,401.232,372.90
May,1858.1121,8747.2,9776.8,526.136,525.45
Jun,2519.8815,11368.0,12012.0,716.040,762.75
Jul,2240.5746,10998.4,13464.0,716.976,806.82
Aug,2246.6940,10796.8,12636.8,676.000,718.68
Sep,1906.6302,8937.6,10084.8,563.472,603.42


In [21]:
cap_tracker = annual_contracted_limit.copy()

def order(a):
    # orders = pd.DataFrame(np.zeros(M, J), index=month, columns=supplier)
    orders = pd.DataFrame(0, index=alloy_demand.transpose().columns.values.tolist(), columns=annual_contracted_limit.columns.values.tolist())
    cap_tracker = annual_contracted_limit.loc[a].copy()
    
    for m in month:
        dem = alloy_demand.loc[m, a]
        sup_prices = unit_cost.loc[a]
        thresh_tracker = contract_upper_threshold.loc[a].copy()
        
        while dem > 0:            
            for s in annual_contracted_limit.columns.values.tolist():
                if cap_tracker[s] == 0:
                    # Prohibit buying from suppliers with depleted stock for the year
                    sup_prices[s] = np.inf

                elif thresh_tracker[s] <= 0:
                    # Update pricing if month threshoold exceeded
                    sup_prices[s] = unit_cost.loc[a, s]*(1+contract_penalties[s])

            s_min = sup_prices.idxmin()

            if thresh_tracker[s_min] > 0: # have not exceeded monthly threshold yet
                # make an order that does not exceed monthly threshold, remaining demand, and remaining annual capacity
                order = min(thresh_tracker[s_min], cap_tracker[s_min], dem)
                # Update trackers
                dem -= order
                cap_tracker[s_min]-=order
                thresh_tracker[s_min]-=order
                orders.loc[m, s_min]+=order

            else: # Monthly theshold exceeded but still best option
                order = min(cap_tracker[s_min], dem)
                # Update trackers
                dem -= order
                cap_tracker[s_min]-=order
                thresh_tracker[s_min]-=order
                orders.loc[m, s_min]+=order
            
    return orders

In [22]:
all_1_base = order('all_1')
all_1_base

,Supp A,Supp B,Supp C,Supp D,Supp E
Init,0.0000,0.0000,0.0000,0.0000,0.0000
Jan,0.0000,615.7134,0.0000,0.0000,1200.0000
Feb,0.0000,1000.0000,0.0000,809.5940,0.0000
Mar,0.0000,0.0000,630.3830,1000.0000,0.0000
Apr,0.0000,0.0000,1469.5302,0.0000,0.0000
May,358.1121,0.0000,1500.0000,0.0000,0.0000
Jun,1500.0000,0.0000,0.0000,0.0000,1019.8815
Jul,0.0000,1000.0000,0.0000,40.5746,1200.0000
Aug,0.0000,0.0000,1246.6940,1000.0000,0.0000
Sep,406.6302,0.0000,1500.0000,0.0000,0.0000


In [23]:
all_2_base=order('all_2')
all_2_base

,Supp A,Supp B,Supp C,Supp D,Supp E
Init,0.0,0,0.0,0.0,0.0
Jan,0.0,2500,0.0,676.0,5000.0
Feb,0.0,0,788.0,7500.0,0.0
Mar,0.0,0,7392.0,0.0,0.0
Apr,0.0,0,6966.4,0.0,0.0
May,1493.6,0,7253.6,0.0,0.0
Jun,10000.0,1368,0.0,0.0,0.0
Jul,998.4,2500,0.0,7500.0,0.0
Aug,10000.0,0,0.0,0.0,796.8
Sep,0.0,0,0.0,3937.6,5000.0


In [24]:
all_3_base=order('all_3')
all_3_base

,Supp A,Supp B,Supp C,Supp D,Supp E
Init,0.0,0.0,0,0.0,0.0
Jan,0.0,3000.0,0,3278.4,1800.0
Feb,0.0,0.0,4124,4500.0,0.0
Mar,1744.0,0.0,6000,0.0,0.0
Apr,7761.6,0.0,0,0.0,0.0
May,8094.4,1682.4,0,0.0,0.0
Jun,0.0,3000.0,2712,4500.0,1800.0
Jul,0.0,0.0,6000,0.0,7464.0
Aug,0.0,336.8,6000,4500.0,1800.0
Sep,0.0,10084.8,0,0.0,0.0


In [25]:
all_4_base=order('all_4')
all_4_base

,Supp A,Supp B,Supp C,Supp D,Supp E
Init,0.000,0.000,0.000,0.000,0.000
Jan,0.000,250.000,0.000,81.184,100.000
Feb,0.000,0.000,163.840,300.000,0.000
Mar,0.000,0.000,444.080,0.000,0.000
Apr,0.000,0.000,401.232,0.000,0.000
May,76.136,0.000,450.000,0.000,0.000
Jun,450.000,250.000,0.000,16.040,0.000
Jul,0.000,8.128,308.848,300.000,100.000
Aug,426.000,250.000,0.000,0.000,0.000
Sep,450.000,13.472,0.000,0.000,100.000


In [26]:
all_5_base=order('all_5')
all_5_base

,Supp A,Supp B,Supp C,Supp D,Supp E
Init,0.00,0.00,0.00,0.00,0.00
Jan,0.00,132.22,0.00,0.00,200.00
Feb,0.00,250.00,0.00,156.80,0.00
Mar,0.00,0.00,140.70,300.00,0.00
Apr,0.00,0.00,372.90,0.00,0.00
May,75.45,0.00,450.00,0.00,0.00
Jun,500.00,62.75,0.00,0.00,200.00
Jul,0.00,250.00,256.82,300.00,0.00
Aug,0.00,0.00,450.00,68.68,200.00
Sep,303.42,0.00,0.00,300.00,0.00


In [27]:
unit_cost = pd.read_excel(file_loc,sheet_name='Unit Cost',header=2, na_values=['NA'], usecols="B:F")
unit_cost = unit_cost.iloc[:5,:]
unit_cost.index=alloy

In [28]:
unit_cost

,Supp A,Supp B,Supp C,Supp D,Supp E
all_1,275.000000,274.166667,274.583333,274.416667,273.750000
all_2,11.250000,10.666667,11.083333,10.833333,10.416667
all_3,2.333333,1.833333,2.166667,2.000000,1.666667
all_4,858.333333,855.000000,857.083333,855.833333,853.333333
all_5,210.833333,204.166667,208.333333,205.833333,202.500000


In [29]:
total_cost = 0
combine = [all_1_base,all_2_base,all_3_base,all_4_base,all_5_base]

In [30]:
for i in range(I):
    for j in range(J):
        value = unit_cost.iloc[i,j]*sum(combine[i].iloc[:,j])
        total_cost += value

In [31]:
total_cost

15868089.687525002